In [1]:
#Loading csv with processed text and csv with feature_vecs (100 word embedding vectors + dummy variables for essay set)
import pandas as pd
import numpy as np
df = pd.read_csv('..\data\interim\processed_text.csv', index_col=0)
X = pd.read_csv('..\data\interim\\feature_vecs.csv', index_col=0)

In [2]:
#Scaling feature_vectors
from sklearn.preprocessing import MinMaxScaler
scaler_x = MinMaxScaler()
X = scaler_x.fit_transform(X)

In [3]:
#Single scaler for all of the domain1_scores despite them having different ranges
scaler_y = MinMaxScaler()

essay_scores = df.loc[:,'domain1_score'].values.reshape(-1,1)
y_scaled = scaler_y.fit_transform(essay_scores)
y_scaled = np.concatenate(y_scaled).flatten()

y = pd.DataFrame({'essay_set':df.essay_set.values, 'scaled':y_scaled})

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=728)

In [ ]:
#Training gradientboostingregressor
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV

params = {'n_estimators':[50, 100, 500, 1000], 'max_depth':[2], 'min_samples_split': [2],
          'learning_rate':[1, 0.1, 0.3, 0.01], 'loss': ['ls']}

gbr = ensemble.GradientBoostingRegressor()
grid = GridSearchCV(gbr, params, cv=3)
grid.fit(X_train, y_train.scaled)

y_pred = grid.predict(X_test)

# summarize the results of the grid search
print(grid.best_score_)
print(grid.best_estimator_)

In [6]:
from sklearn import ensemble
model = ensemble.GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=2, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=500, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [7]:
model.fit(X_train,y_train.scaled)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=2, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=500, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [8]:
#Scaled original and predicted values
y_pred_scaled = model.predict(X_test)
y_test_scaled = y_test.scaled.values

#Reversing transformation for original and predicted
y_test_unscaled = scaler_y.inverse_transform(y_test_scaled.reshape(-1,1))
y_pred_unscaled = scaler_y.inverse_transform(y_pred_scaled.reshape(-1,1))

#Needed reshaping
y_test_unscaled = y_test_unscaled.reshape(-1,)
y_pred_unscaled = y_pred_unscaled.reshape(-1,)

#Creating dataframe, updating columns, and saving to interim data folder for later exploration
predictions = pd.DataFrame({'essay_set': y_test.essay_set, 'pred_scaled': y_pred_scaled,
                            'pred_unscaled': y_pred_unscaled,'orig_scaled': y_test_scaled,
                            'orig_unscaled': y_test_unscaled}, index=y_test.index)
predictions.to_csv('..\data\interim\single_scaler_results.csv')

In [9]:
predictions

,essay_set,pred_scaled,pred_unscaled,orig_scaled,orig_unscaled
6970,4,0.035342,2.120522,0.050000,3.0
6711,4,0.026312,1.578712,0.016667,1.0
9297,6,0.066861,4.011646,0.050000,3.0
4826,3,0.031160,1.869597,0.033333,2.0
10903,7,0.276846,16.610752,0.233333,14.0
76,1,0.156857,9.411439,0.166667,10.0
12868,8,0.614425,36.865528,0.500000,30.0
2342,2,0.047917,2.874997,0.066667,4.0
5842,4,0.048679,2.920767,0.050000,3.0
10116,6,0.030558,1.833497,0.033333,2.0
